In [1]:
import tensorflow as tf 
import tensorflow_datasets as tfds
import numpy as np
import jax.numpy as jnp
from jax import grad, jit, vmap
import jax
import flax.nnx
import optax
from functools import partial

2025-03-15 15:14:46.072451: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742051686.085915   32036 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742051686.089943   32036 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742051686.101860   32036 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742051686.101873   32036 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742051686.101875   32036 computation_placer.cc:177] computation placer alr

In [2]:
class cnn(flax.nnx.Module):
    def __init__(self, 
                 regularizer = 1e-4,
                 *args, 
                 **kwargs):
      
        self.rngs = flax.nnx.Rngs(3)
        self.regularizer = regularizer
        self.loss = 0.
        self.layers = []
        
        self.conv1 = flax.nnx.Conv(1, 32, (3, 3), strides=2, padding='SAME', rngs=self.rngs) # [N, 16, 16,32]
        self.conv2 = flax.nnx.Conv(32, 64, (3, 3), strides=2, padding='SAME', rngs=self.rngs) # [N, 8, 8, 64]
        self.conv3 = flax.nnx.Conv(64, 128, (3, 3), strides=2, padding='SAME', rngs=self.rngs) # [N, 4, 4, 128]
        self.conv4 = flax.nnx.Conv(128, 256, (3, 3), strides=2, padding='SAME', rngs=self.rngs) # [N, 2, 2, 256]
        self.conv5 = flax.nnx.Conv(256, 10, (3, 3), strides=2, padding='SAME', rngs=self.rngs) # [N, 1, 1, 10]
    
        self.layers = [self.conv1, self.conv2, self.conv3, self.conv4, self.conv5]
    
    @flax.nnx.jit
    def __call__(self, x):
        conv1 = flax.nnx.relu(self.conv1(x))
        conv2 = flax.nnx.relu(self.conv2(conv1))
        conv3 = flax.nnx.relu(self.conv3(conv2))
        conv4 = flax.nnx.relu(self.conv4(conv3))
        out = self.conv5(conv4)
        
        return out.reshape(-1, 10)
    
    # addtional loss
    def kernel_bias_L2regularization(self):
        loss = 0 # 不能用self.loss，因為tree_util的結果不給儲存
        for layer in self.layers:
            # weights regularization
            loss += (jax.tree_util.tree_leaves(layer.kernel)[0].sum()) ** 2 
            loss += (jax.tree_util.tree_leaves(layer.bias)[0].sum()) ** 2 
        return loss * self.regularizer
    
model = cnn()
# flax.nnx.display(model)


In [3]:
# Load MNIST dataset
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

# Define a function to normalize the images
def normalize_img(image, label):
    return tf.cast(image, tf.float32) / 255.0, label

# Prepare the training dataset
ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE).repeat()
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(32, drop_remainder=True)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

# Prepare the test dataset
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE).repeat()
ds_test = ds_test.batch(32, drop_remainder=True)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

# initialize training and test dataset iterator
tr_iter = iter(ds_train)
ts_iter = iter(ds_test)


2025-03-15 15:14:50.813616: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
I0000 00:00:1742051690.813749   32036 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5489 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1060 6GB, pci bus id: 0000:01:00.0, compute capability: 6.1


In [4]:
###! 不知道為什麼，loss function最前面一定要放model，不然會報錯。例如放(images,label,model)，把model放最後一個就會出錯
def loss_fn(model, x, y):
    y_hat_logits = model(jnp.array(x))
    ce = optax.losses.softmax_cross_entropy_with_integer_labels(logits=y_hat_logits, labels=jnp.array(y)).mean()
    reg = model.kernel_bias_L2regularization()
    return ce + reg, y_hat_logits

# @flax.nnx.jit
def update_weights(model, opt, tr_iter, innerSteps=100):
    for innerStep in range(innerSteps):
        pics, labels = next(tr_iter)
        # 如果loss function有回傳loss以外的東西就要把has_aux打開，
        # value_and_grad的回傳規則是 [(loss_fn Arg1, loss_fn Arg2, ...), gradient]
        grad_fn = flax.nnx.value_and_grad(loss_fn, has_aux=True)
        (loss, logits), grads = grad_fn(model, pics, labels)
        opt.update(grads)
    return loss, logits
    


In [ ]:
learningRate = 1e-4
trainingStep = 50
opt = flax.nnx.Optimizer(model, optax.adamw(learningRate))

# 如果loss function有回傳loss以外的東西就要把has_aux打開，
# value_and_grad的回傳規則是 [(loss_fn Arg1, loss_fn Arg2, ...), gradient]
grad_fn = flax.nnx.value_and_grad(loss_fn, has_aux=False)

for step in range(trainingStep):
    # optimizing model
    loss, logits = update_weights(model, opt, tr_iter)
    # using test dataset to validate the model with accuracy
    pics, labels = next(ts_iter)
    loss, logits = loss_fn(model, pics, labels)
    accuracy = jnp.mean(jnp.argmax(logits, axis=1) == jnp.array(labels))
    
    print("step {} loss:{} val acc:{}".format(step * 100, loss, accuracy))





2025-03-15 15:14:51.347629: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:387] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 8388608


step 0 loss:1.5701472759246826 val acc:0.8125
step 100 loss:0.6020104289054871 val acc:0.875
step 200 loss:0.3363880217075348 val acc:0.90625
step 300 loss:0.3578474819660187 val acc:0.875
step 400 loss:0.5462193489074707 val acc:0.90625
step 500 loss:0.09150280803442001 val acc:1.0
step 600 loss:0.3464528024196625 val acc:0.90625
step 700 loss:0.18859736621379852 val acc:0.9375
step 800 loss:0.0718141421675682 val acc:0.96875
step 900 loss:0.1552731990814209 val acc:0.96875
step 1000 loss:0.17164109647274017 val acc:0.9375
step 1100 loss:0.3645014464855194 val acc:0.90625
step 1200 loss:0.06559634208679199 val acc:1.0
step 1300 loss:0.15293672680854797 val acc:0.9375
step 1400 loss:0.07625730335712433 val acc:1.0
step 1500 loss:0.21861226856708527 val acc:0.90625
step 1600 loss:0.05607498809695244 val acc:0.96875
step 1700 loss:0.05368279665708542 val acc:1.0
step 1800 loss:0.21398814022541046 val acc:0.9375
step 1900 loss:0.06544697284698486 val acc:0.96875
step 2000 loss:0.134059101